In [2]:
# Notebook: 02-Model-Training-GNN-PyTorch-4.ipynb (Extended for IP & Multi-Task)
# =============================================================================
# This notebook demonstrates multi-task GNN training on processed GNN data
# that includes user, business, and IP nodes, plus masks for train/val/test.
# Now using synergy-based IP labels from your new data pipeline,
# and a correct steps_per_epoch formula for mini-batching (if desired).

import os
import math
import json
import numpy as np

import torch
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, SAGEConv, Linear

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cpu


In [3]:
# --------------------------------------------------------------------------------
# Cell 2: Configuration
# --------------------------------------------------------------------------------
# Path to synergy-based data (already preprocessed into .npy files)
PROCESSED_DIR = (
    "/Users/harshil/Developer/"
    "GitHub_Repos/VeriShield-AI-Financial-Verification-Platform/"
    "verishield_ml_experiments/data_generators/data-v1/high_fraud/processed_gnn"
)

# Multi-task toggles
MULTI_TASK = True          # or False if only user classification
IP_CLASSIFICATION = True   # classify IP nodes?

# GNN Hyperparams
HIDDEN_DIM = 64
LEARNING_RATE = 1e-3
EPOCHS = 200

# BATCH-SIZE & STEPS_PER_EPOCH:
#  - If you do mini-batch training, steps_per_epoch = ceil(train_size / BATCH_SIZE).
#  - If you do full-batch (the code currently does a single forward on the entire graph),
#    then steps_per_epoch=1 is typical. 
BATCH_SIZE = 1024   # example batch size if you do mini-batching
STEPS_PER_EPOCH = 1 # we'll override this later once we load train masks, if we want

PRINT_EVERY = 1

print(f"Processed GNN data at: {PROCESSED_DIR}")
print(f"MULTI_TASK={MULTI_TASK}, IP_CLASSIFICATION={IP_CLASSIFICATION}")
print(f"HIDDEN_DIM={HIDDEN_DIM}, LR={LEARNING_RATE}, EPOCHS={EPOCHS}, BATCH_SIZE={BATCH_SIZE}")
print(f"Initial STEPS_PER_EPOCH={STEPS_PER_EPOCH} (may override after loading data)")

Processed GNN data at: /Users/harshil/Developer/GitHub_Repos/VeriShield-AI-Financial-Verification-Platform/verishield_ml_experiments/data_generators/data-v1/high_fraud/processed_gnn
MULTI_TASK=True, IP_CLASSIFICATION=True
HIDDEN_DIM=64, LR=0.001, EPOCHS=200, BATCH_SIZE=1024
Initial STEPS_PER_EPOCH=1 (may override after loading data)


In [4]:
# --------------------------------------------------------------------------------
# Cell 3: Loading .npy Arrays
# --------------------------------------------------------------------------------
def load_npy(filename):
    path = os.path.join(PROCESSED_DIR, filename)
    arr = np.load(path)
    print(f"Loaded {filename}, shape={arr.shape}, dtype={arr.dtype}")
    return arr

user_features  = torch.from_numpy(load_npy("user_features.npy")).float()
user_labels    = torch.from_numpy(load_npy("user_labels.npy")).long()
biz_features   = torch.from_numpy(load_npy("biz_features.npy")).float()
biz_labels     = torch.from_numpy(load_npy("biz_labels.npy")).long()
ip_features    = torch.from_numpy(load_npy("ip_features.npy")).float()
ip_labels      = torch.from_numpy(load_npy("ip_labels.npy")).long()

edge_user_user = torch.from_numpy(load_npy("edge_user_user.npy")).long()
edge_user_biz  = torch.from_numpy(load_npy("edge_user_biz.npy")).long()
edge_user_ip   = torch.from_numpy(load_npy("edge_user_ip.npy")).long()

# Load train/val/test masks
train_mask_users = None
val_mask_users   = None
test_mask_users  = None
try:
    train_mask_users = torch.from_numpy(load_npy("train_mask_users.npy")).bool()
    val_mask_users   = torch.from_numpy(load_npy("val_mask_users.npy")).bool()
    test_mask_users  = torch.from_numpy(load_npy("test_mask_users.npy")).bool()
except FileNotFoundError:
    print("No user masks found; will do full-batch on users if MULTI_TASK=False.")

train_mask_biz = None
val_mask_biz   = None
test_mask_biz  = None
if MULTI_TASK:
    try:
        train_mask_biz = torch.from_numpy(load_npy("train_mask_biz.npy")).bool()
        val_mask_biz   = torch.from_numpy(load_npy("val_mask_biz.npy")).bool()
        test_mask_biz  = torch.from_numpy(load_npy("test_mask_biz.npy")).bool()
    except FileNotFoundError:
        print("No business masks found; can't do multi-task effectively without them.")

train_mask_ip = None
val_mask_ip   = None
test_mask_ip  = None
if IP_CLASSIFICATION:
    try:
        train_mask_ip = torch.from_numpy(load_npy("train_mask_ip.npy")).bool()
        val_mask_ip   = torch.from_numpy(load_npy("val_mask_ip.npy")).bool()
        test_mask_ip  = torch.from_numpy(load_npy("test_mask_ip.npy")).bool()
    except FileNotFoundError:
        print("No IP masks found; IP classification won't be possible without them.")

# Optionally load metadata.json
meta_path = os.path.join(PROCESSED_DIR, "metadata.json")
if os.path.isfile(meta_path):
    with open(meta_path, "r") as f:
        metadata = json.load(f)
    print("\nmetadata.json contents:")
    for k,v in metadata.items():
        print(f"  {k}: {v}")

Loaded user_features.npy, shape=(50000, 7), dtype=float32
Loaded user_labels.npy, shape=(50000,), dtype=int64
Loaded biz_features.npy, shape=(10000, 3), dtype=float32
Loaded biz_labels.npy, shape=(10000,), dtype=int64
Loaded ip_features.npy, shape=(8000, 0), dtype=float32
Loaded ip_labels.npy, shape=(8000,), dtype=int64
Loaded edge_user_user.npy, shape=(2, 4918), dtype=int64
Loaded edge_user_biz.npy, shape=(2, 110731), dtype=int64
Loaded edge_user_ip.npy, shape=(2, 50000), dtype=int64
Loaded train_mask_users.npy, shape=(50000,), dtype=bool
Loaded val_mask_users.npy, shape=(50000,), dtype=bool
Loaded test_mask_users.npy, shape=(50000,), dtype=bool
Loaded train_mask_biz.npy, shape=(10000,), dtype=bool
Loaded val_mask_biz.npy, shape=(10000,), dtype=bool
Loaded test_mask_biz.npy, shape=(10000,), dtype=bool
Loaded train_mask_ip.npy, shape=(8000,), dtype=bool
Loaded val_mask_ip.npy, shape=(8000,), dtype=bool
Loaded test_mask_ip.npy, shape=(8000,), dtype=bool

metadata.json contents:
  scenar

In [5]:
# --------------------------------------------------------------------------------
# Cell 4: Adjust STEPS_PER_EPOCH if Doing Mini-Batch
# --------------------------------------------------------------------------------
# The code below still does "full-batch" training, but here's how you'd compute steps_per_epoch 
# if you truly do mini-batching at the node level (not shown in detail here).
# We'll pick 'user' training set size to define steps_per_epoch, for instance:

if train_mask_users is not None:
    train_user_count = train_mask_users.sum().item()
    if train_user_count > 0:
        STEPS_PER_EPOCH = math.ceil(train_user_count / BATCH_SIZE)
        print(f"Adjusted STEPS_PER_EPOCH = {STEPS_PER_EPOCH} (from user train count={train_user_count}, batch_size={BATCH_SIZE})")
    else:
        # fallback to 1 if no users in train set
        STEPS_PER_EPOCH = 1
else:
    # full-batch
    STEPS_PER_EPOCH = 1

print(f"Final STEPS_PER_EPOCH: {STEPS_PER_EPOCH}")

Adjusted STEPS_PER_EPOCH = 35 (from user train count=35000, batch_size=1024)
Final STEPS_PER_EPOCH: 35


In [6]:
# --------------------------------------------------------------------------------
# Cell 5: Build a HeteroData Graph (Users, Businesses, IPs)
# --------------------------------------------------------------------------------
data = HeteroData()

# user nodes
data['user'].x = user_features
data['user'].y = user_labels
if train_mask_users is not None:
    data['user'].train_mask = train_mask_users
    data['user'].val_mask   = val_mask_users
    data['user'].test_mask  = test_mask_users

# business nodes
data['business'].x = biz_features
data['business'].y = biz_labels
if MULTI_TASK and train_mask_biz is not None:
    data['business'].train_mask = train_mask_biz
    data['business'].val_mask   = val_mask_biz
    data['business'].test_mask  = test_mask_biz

# ip nodes
data['ip'].x = ip_features
data['ip'].y = ip_labels
if IP_CLASSIFICATION and train_mask_ip is not None:
    data['ip'].train_mask  = train_mask_ip
    data['ip'].val_mask    = val_mask_ip
    data['ip'].test_mask   = test_mask_ip

# Edges
data[('user','user_user','user')].edge_index = edge_user_user
data[('user','user_business','business')].edge_index = edge_user_biz
data[('user','user_ip','ip')].edge_index = edge_user_ip

# Reverse edges for message passing in both directions
rev_user_biz = torch.flip(edge_user_biz, dims=[0])
data[('business','rev_user_business','user')].edge_index = rev_user_biz

rev_user_ip = torch.flip(edge_user_ip, dims=[0])
data[('ip','rev_user_ip','user')].edge_index = rev_user_ip

print("\nHeteroData object created with node_types:", list(data.node_types))
print("Edge types:", list(data.edge_types))

# Move entire data to GPU
data = data.to(device)


HeteroData object created with node_types: ['user', 'business', 'ip']
Edge types: [('user', 'user_user', 'user'), ('user', 'user_business', 'business'), ('user', 'user_ip', 'ip'), ('business', 'rev_user_business', 'user'), ('ip', 'rev_user_ip', 'user')]


In [7]:
# --------------------------------------------------------------------------------
# Cell 6: Define a GNN Model (Supports Multi-Edges & Optional Multi-Task)
# --------------------------------------------------------------------------------
class FraudGNN(torch.nn.Module):
    def __init__(self, hidden_dim=64, user_out=2, biz_out=2, ip_out=2,
                 multi_task=False, ip_task=False):
        """
        multi_task=True => We'll produce heads for user + business
        ip_task=True    => We'll also produce a head for IP

        Otherwise, we do a single user head only.
        """
        super().__init__()
        self.multi_task = multi_task
        self.ip_task = ip_task

        # Single HeteroConv for all relevant edges:
        self.conv1 = HeteroConv({
            ('user','user_user','user'): SAGEConv((-1, -1), hidden_dim),
            ('user','user_business','business'): SAGEConv((-1, -1), hidden_dim),
            ('business','rev_user_business','user'): SAGEConv((-1, -1), hidden_dim),
            ('user','user_ip','ip'): SAGEConv((-1, -1), hidden_dim),
            ('ip','rev_user_ip','user'): SAGEConv((-1, -1), hidden_dim),
        }, aggr='mean')

        self.conv2 = HeteroConv({
            ('user','user_user','user'): SAGEConv((-1, -1), hidden_dim),
            ('user','user_business','business'): SAGEConv((-1, -1), hidden_dim),
            ('business','rev_user_business','user'): SAGEConv((-1, -1), hidden_dim),
            ('user','user_ip','ip'): SAGEConv((-1, -1), hidden_dim),
            ('ip','rev_user_ip','user'): SAGEConv((-1, -1), hidden_dim),
        }, aggr='mean')

        # Final linear heads
        self.user_lin = Linear(hidden_dim, user_out)

        if multi_task:
            self.business_lin = Linear(hidden_dim, biz_out)
        else:
            self.business_lin = None

        if ip_task:
            self.ip_lin = Linear(hidden_dim, ip_out)
        else:
            self.ip_lin = None

    def forward(self, x_dict, edge_index_dict):
        # 1st layer
        x_dict = self.conv1(x_dict, edge_index_dict)
        for ntype, x in x_dict.items():
            x_dict[ntype] = F.relu(x)

        # 2nd layer
        x_dict = self.conv2(x_dict, edge_index_dict)
        for ntype, x in x_dict.items():
            x_dict[ntype] = F.relu(x)

        return x_dict

    def forward_user(self, emb_dict):
        return self.user_lin(emb_dict['user'])

    def forward_business(self, emb_dict):
        if self.business_lin is None:
            return None
        return self.business_lin(emb_dict['business'])

    def forward_ip(self, emb_dict):
        if self.ip_lin is None:
            return None
        return self.ip_lin(emb_dict['ip'])

model = FraudGNN(
    hidden_dim=HIDDEN_DIM,
    user_out=2,         # user classification: legit/fraud
    biz_out=2,          # business classification
    ip_out=2,           # IP classification
    multi_task=MULTI_TASK,
    ip_task=IP_CLASSIFICATION
).to(device)

print(model)

FraudGNN(
  (conv1): HeteroConv(num_relations=5)
  (conv2): HeteroConv(num_relations=5)
  (user_lin): Linear(64, 2, bias=True)
  (business_lin): Linear(64, 2, bias=True)
  (ip_lin): Linear(64, 2, bias=True)
)


In [8]:
# --------------------------------------------------------------------------------
# Cell 7: Training Setup (Multi-Task or Single)
# --------------------------------------------------------------------------------
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

def train_step():
    """
    One train step on the entire graph (masked to train sets).
    If truly doing mini-batch, you'd sample subgraph or node-batches here
    instead of passing the entire data each step.
    """
    model.train()
    optimizer.zero_grad()

    emb_dict = model(data.x_dict, data.edge_index_dict)

    # User classification
    user_logits = model.forward_user(emb_dict)
    train_mask_u = data['user'].train_mask if 'train_mask' in data['user'] else None
    user_loss = 0.
    if train_mask_u is not None:
        masked_logits = user_logits[train_mask_u]
        masked_labels = data['user'].y[train_mask_u]
        user_loss = F.cross_entropy(masked_logits, masked_labels)
    else:
        user_loss = F.cross_entropy(user_logits, data['user'].y)

    # Business classification (multi-task)
    biz_loss = 0.
    if model.multi_task and 'train_mask' in data['business']:
        biz_logits = model.forward_business(emb_dict)
        train_mask_b = data['business'].train_mask
        masked_biz_logits = biz_logits[train_mask_b]
        masked_biz_labels = data['business'].y[train_mask_b]
        biz_loss = F.cross_entropy(masked_biz_logits, masked_biz_labels)

    # IP classification (ip_task)
    ip_loss = 0.
    if model.ip_task and 'train_mask' in data['ip']:
        ip_logits = model.forward_ip(emb_dict)
        train_mask_i = data['ip'].train_mask
        masked_ip_logits = ip_logits[train_mask_i]
        masked_ip_labels = data['ip'].y[train_mask_i]
        ip_loss = F.cross_entropy(masked_ip_logits, masked_ip_labels)

    total_loss = user_loss + biz_loss + ip_loss
    total_loss.backward()
    optimizer.step()
    return float(total_loss)

@torch.no_grad()
def evaluate_user(mask_name='val_mask'):
    """Compute user accuracy on val/test mask."""
    model.eval()
    emb_dict = model(data.x_dict, data.edge_index_dict)
    user_logits = model.forward_user(emb_dict)
    if mask_name not in data['user']:
        preds = user_logits.argmax(dim=-1)
        correct = (preds == data['user'].y).sum()
        return float(correct / len(data['user'].y))
    mask = data['user'][mask_name]
    logits_masked = user_logits[mask]
    labels_masked = data['user'].y[mask]
    preds = logits_masked.argmax(dim=-1)
    correct = (preds == labels_masked).sum()
    return float(correct / mask.sum())

@torch.no_grad()
def evaluate_biz(mask_name='val_mask'):
    """Compute business accuracy if multi-task is enabled."""
    if not model.multi_task or mask_name not in data['business']:
        return 0.0
    model.eval()
    emb_dict = model(data.x_dict, data.edge_index_dict)
    biz_logits = model.forward_business(emb_dict)
    mask = data['business'][mask_name]
    logits_masked = biz_logits[mask]
    labels_masked = data['business'].y[mask]
    preds = logits_masked.argmax(dim=-1)
    correct = (preds == labels_masked).sum()
    return float(correct / mask.sum())

@torch.no_grad()
def evaluate_ip(mask_name='val_mask'):
    """Compute IP accuracy if IP classification is enabled."""
    if not model.ip_task or mask_name not in data['ip']:
        return 0.0
    model.eval()
    emb_dict = model(data.x_dict, data.edge_index_dict)
    ip_logits = model.forward_ip(emb_dict)
    mask = data['ip'][mask_name]
    logits_masked = ip_logits[mask]
    labels_masked = data['ip'].y[mask]
    preds = logits_masked.argmax(dim=-1)
    correct = (preds == labels_masked).sum()
    return float(correct / mask.sum())

In [9]:
# --------------------------------------------------------------------------------
# Cell 8: Main Training Loop
# --------------------------------------------------------------------------------
for epoch in range(1, EPOCHS + 1):
    avg_loss = 0.0
    for step in range(STEPS_PER_EPOCH):
        # If truly mini-batching, you'd do subgraph sampling here.
        loss_val = train_step()
        avg_loss += loss_val

    if STEPS_PER_EPOCH > 1:
        avg_loss /= STEPS_PER_EPOCH

    # Evaluate on validation masks
    val_acc_user = evaluate_user('val_mask')
    val_acc_biz  = evaluate_biz('val_mask') if MULTI_TASK else 0.0
    val_acc_ip   = evaluate_ip('val_mask') if IP_CLASSIFICATION else 0.0

    if epoch % PRINT_EVERY == 0:
        print(f"Epoch {epoch}/{EPOCHS} => loss={avg_loss:.4f}, "
              f"user_val_acc={val_acc_user:.4f}, biz_val_acc={val_acc_biz:.4f}, ip_val_acc={val_acc_ip:.4f}")

# Final test evaluation
print("\nFinal Evaluations:")
user_test_acc = evaluate_user('test_mask')
print(f"User Test Accuracy: {user_test_acc:.4f}")
if MULTI_TASK:
    biz_test_acc = evaluate_biz('test_mask')
    print(f"Business Test Accuracy: {biz_test_acc:.4f}")
if IP_CLASSIFICATION:
    ip_test_acc = evaluate_ip('test_mask')
    print(f"IP Test Accuracy: {ip_test_acc:.4f}")

Epoch 1/200 => loss=1.6061, user_val_acc=0.6801, biz_val_acc=0.9227, ip_val_acc=0.6417
Epoch 2/200 => loss=1.4833, user_val_acc=0.6801, biz_val_acc=0.9227, ip_val_acc=0.6500
Epoch 3/200 => loss=1.4694, user_val_acc=0.6867, biz_val_acc=0.9227, ip_val_acc=0.6533
Epoch 4/200 => loss=1.4574, user_val_acc=0.6951, biz_val_acc=0.9227, ip_val_acc=0.6492
Epoch 5/200 => loss=1.4468, user_val_acc=0.7029, biz_val_acc=0.9227, ip_val_acc=0.6492
Epoch 6/200 => loss=1.4379, user_val_acc=0.7057, biz_val_acc=0.9227, ip_val_acc=0.6467
Epoch 7/200 => loss=1.4309, user_val_acc=0.7076, biz_val_acc=0.9227, ip_val_acc=0.6408
Epoch 8/200 => loss=1.4246, user_val_acc=0.7073, biz_val_acc=0.9227, ip_val_acc=0.6392
Epoch 9/200 => loss=1.4183, user_val_acc=0.7068, biz_val_acc=0.9227, ip_val_acc=0.6442
Epoch 10/200 => loss=1.4119, user_val_acc=0.7068, biz_val_acc=0.9227, ip_val_acc=0.6417
Epoch 11/200 => loss=1.4053, user_val_acc=0.7068, biz_val_acc=0.9227, ip_val_acc=0.6392
Epoch 12/200 => loss=1.3982, user_val_acc

In [10]:
# --------------------------------------------------------------------------------
# Cell 9: Wrap-Up
# --------------------------------------------------------------------------------
print("""
Done!
We've built a multi-edge, multi-task GNN using PyTorch Geometric:
- Node types: user, business, ip
- Edge types: user->user, user->business, user->ip (+ reversed edges)
- Multi-task heads for user, business, and IP classification
- 'steps_per_epoch' formula demonstrated for mini-batch. 
- Final accuracy results printed for each node type.
""")


Done!
We've built a multi-edge, multi-task GNN using PyTorch Geometric:
- Node types: user, business, ip
- Edge types: user->user, user->business, user->ip (+ reversed edges)
- Multi-task heads for user, business, and IP classification
- 'steps_per_epoch' formula demonstrated for mini-batch. 
- Final accuracy results printed for each node type.

